In [1]:
import wave
import fnmatch
import os
import pandas as pd
import struct

import numpy as np

normal_df = pd.DataFrame()
directory = './heartbeat-sounds/set_b/'

for filename in os.listdir(directory):
    if filename.endswith(".wav") and fnmatch.fnmatch(filename, '*normal__*'):
        FNAME = os.path.join(directory, filename)
        print(FNAME)
        
        f = wave.open(FNAME)
        frames = f.readframes(-1)
        samples = struct.unpack('h'*f.getnframes(), frames)
               
        temp_df = pd.DataFrame(list(samples))
        normal_df = normal_df.append(temp_df.T,ignore_index=True)
        
        continue
    else:
        continue

./heartbeat-sounds/set_b/normal__103_1305031931979_B.wav
./heartbeat-sounds/set_b/normal__103_1305031931979_D1.wav
./heartbeat-sounds/set_b/normal__103_1305031931979_D2.wav
./heartbeat-sounds/set_b/normal__103_1305031931979_D3.wav
./heartbeat-sounds/set_b/normal__106_1306776721273_B1.wav
./heartbeat-sounds/set_b/normal__106_1306776721273_C1.wav
./heartbeat-sounds/set_b/normal__106_1306776721273_C2.wav
./heartbeat-sounds/set_b/normal__106_1306776721273_D1.wav
./heartbeat-sounds/set_b/normal__106_1306776721273_D2.wav
./heartbeat-sounds/set_b/normal__107_1305654946865_C1.wav
./heartbeat-sounds/set_b/normal__109_1305653646620_C.wav
./heartbeat-sounds/set_b/normal__113_1306244002866_D.wav
./heartbeat-sounds/set_b/normal__115_1306259437619_C.wav
./heartbeat-sounds/set_b/normal__117_1306262456650_B.wav
./heartbeat-sounds/set_b/normal__125_1306332456645_B.wav
./heartbeat-sounds/set_b/normal__126_1306777102824_B.wav
./heartbeat-sounds/set_b/normal__126_1306777102824_C.wav
./heartbeat-sounds/set

In [2]:
print(normal_df.shape)

(200, 58720)


In [3]:
x = pd.DataFrame()
x = normal_df.T[:44100]
x_train = np.array(x.T[:160])
print(x_train.shape)

(160, 44100)


In [4]:
x_train = x_train.reshape(x_train.shape[0], 1, 44100)
print(x_train.shape)

(160, 1, 44100)


In [5]:
x_test = np.array(x.T[160:180])
print(x_test.shape)

(20, 44100)


In [6]:
x_validate = np.array(x.T[180:])
print(x_validate.shape)

(20, 44100)


In [7]:
murmur_df = pd.DataFrame()
for filename in os.listdir(directory):
    if filename.endswith(".wav") and fnmatch.fnmatch(filename, '*murmur__*'):
        FNAME = os.path.join(directory, filename)
        print(FNAME)
    
        f = wave.open(FNAME)
        frames = f.readframes(-1)
        samples = struct.unpack('h'*f.getnframes(), frames)
        
        temp_df = pd.DataFrame(list(samples))
        murmur_df = murmur_df.append(temp_df.T,ignore_index=True)
        continue
    else:
        continue

./heartbeat-sounds/set_b/murmur__112_1306243000964_A.wav
./heartbeat-sounds/set_b/murmur__112_1306243000964_B.wav
./heartbeat-sounds/set_b/murmur__112_1306243000964_D.wav
./heartbeat-sounds/set_b/murmur__116_1306258689913_A.wav
./heartbeat-sounds/set_b/murmur__116_1306258689913_C.wav
./heartbeat-sounds/set_b/murmur__116_1306258689913_D.wav
./heartbeat-sounds/set_b/murmur__122_1306325762831_C.wav
./heartbeat-sounds/set_b/murmur__122_1306325762831_D.wav
./heartbeat-sounds/set_b/murmur__156_1306936373241_B.wav
./heartbeat-sounds/set_b/murmur__160_1307100683334_A.wav
./heartbeat-sounds/set_b/murmur__160_1307100683334_B.wav
./heartbeat-sounds/set_b/murmur__161_1307101199321_A.wav
./heartbeat-sounds/set_b/murmur__162_1307101835989_A.wav
./heartbeat-sounds/set_b/murmur__162_1307101835989_B.wav
./heartbeat-sounds/set_b/murmur__164_1307106095995_B.wav
./heartbeat-sounds/set_b/murmur__164_1307106095995_C.wav
./heartbeat-sounds/set_b/murmur__165_1307109069581_C.wav
./heartbeat-sounds/set_b/murmur

In [8]:
print(murmur_df.shape)

(66, 96640)


In [9]:
y = pd.DataFrame()
y = murmur_df.T[:44100]
y_train = np.array(y.T[:54])
print(y_train.shape)

(54, 44100)


In [10]:
y_test = np.array(y.T[54:60])
print(y_test.shape)

(6, 44100)


In [11]:
y_validate = np.array(y.T[60:])
print(y_validate.shape)

(6, 44100)


In [12]:
import keras
import kapre
from keras.models import Sequential
from kapre.time_frequency import Spectrogram
import pylab

Using TensorFlow backend.


In [14]:
model = Sequential()
model.add(Spectrogram(n_dft=512, n_hop=256, input_shape=x_train.shape, 
          return_decibel_spectrogram=True, power_spectrogram=2.0, 
          trainable_kernel=False, name='static_stft'))
model.add(keras.layers.Convolution2D(32, (3, 3), name='conv1', activation='relu'))
model.add(keras.layers.MaxPooling2D((25, 17)))
model.add(keras.layers.Convolution2D(32, (10, 10), name='conv2', activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation='softmax'))
model.summary(line_length=80, positions=[.33, .65, .8, 1.])

AssertionError: Hey! The input is too short!